# Read HDF files from results database
This notebook shows how to read, visualize and process openmc results. In addition, it shows how to store them in an HDF file for the `\results_database\` folder of a given benchmark. To do that we will take advantage of the `\example results\` and `\example_database\` folders. In the former we stored an example openmc `statepoint` file (`example_statepoint.100.h5`) to read. In the latter we will store our new `my_openmc_results.h5` file with the results rearranged consistently with the other HDF files already present in the `\results_database\` folders.

In [1]:
import openmc_sinbad_benchmarks as osb
import numpy as np

Read openmc `statepoint` file:

In [2]:
openmc_results = osb.ResultsFromOpenmc(statepoint_file='example_statepoint.100.h5', path='example_results')

Unfortunately we need a bit of postoprocessing before storing them in a new HDF file. Most likely we need to divide the tally by the geometry filter dimensions (e.g. surface area, cell volume, mesh element volume etc.) but other steps might be necessary, depending on the tally. Here we have the volume of the cells.

In [3]:
_foil_volume = .1 * 1.8**2/4 * np.pi
volumes_onaxis1 = np.concatenate((np.ones(5), np.ones(4)*2, np.ones(4)*3)) * _foil_volume

openmc_results.get_tally_dataframe(tally_name='rr_onaxis1_nb93', normalize_over=None)

,cell,particle,energyfunction,nuclide,score,mean,std. dev.
0,135,neutron,c0eb4a8954a405,total,flux,3.109336e-04,4.155798e-08
1,158,neutron,c0eb4a8954a405,total,flux,3.567396e-05,1.391516e-08
2,181,neutron,c0eb4a8954a405,total,flux,1.273627e-05,8.185310e-09
3,204,neutron,c0eb4a8954a405,total,flux,6.095352e-06,5.048533e-09
4,602,neutron,c0eb4a8954a405,total,flux,4.710290e-06,4.203105e-09
5,239,neutron,c0eb4a8954a405,total,flux,6.266556e-06,6.904215e-09
6,262,neutron,c0eb4a8954a405,total,flux,1.846997e-06,4.706767e-09
7,285,neutron,c0eb4a8954a405,total,flux,5.232285e-07,2.297467e-09
8,308,neutron,c0eb4a8954a405,total,flux,1.450764e-07,1.102283e-09
9,331,neutron,c0eb4a8954a405,total,flux,6.181171e-08,9.447441e-10


I is possible to extract some useful information about the code and the simulation settings: 

In [4]:
version = openmc_results.get_openmc_version
batches = openmc_results.get_batches
ppbatch = openmc_results.get_particles_per_batch

print(f'Code version: {version}')
print(f'Batches: {batches}')
print(f'Particles per batch: {ppbatch}')

Code version: (0, 13, 3)
Batches: 100
Particles per batch: 1.00e+08


## Store results in a database-compatible HDF file
It is possible to store the postprocessed tally in a HDF file compatible with the other files already present in the `\results_database\` folders of the benchmarks available in `\model\`. Of course some information need to be given:
- The name of the HDF file (either existing or to create) to store the results in
- The name of the tally (should not change from the name in the `statepoint` file or, at least, it has to be consistent with the name present in the other HDF files in the database)

Additional:
- Normalization for postprocessing (e.g. cell volume)
- Name of the nuclear data library used (strongly recommended)
- When the simulation has been run
- Where it has been run

In [5]:
xaxis_list = ['0.25', '12.95', '25.95', '38.65', '43.82', '46.35', 
              '53.3', '60.05', '66.9', '73.9', '80.6', '87.25', '91.65']
openmc_results.tally_to_hdf(tally_name='rr_onaxis1_nb93', 
                            normalize_over=volumes_onaxis1, 
                            xs_library='FENDL-3.2b', 
                            xaxis_name='Shield depth (cm)',
                            xaxis_list=xaxis_list,
                            path_to_database='example_database', 
                            when='November 2023', where='MIT-PSFC')

/home/segantin/miniconda3/envs/baby_neutronics/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'Shield depth (cm)'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/segantin/miniconda3/envs/baby_neutronics/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'std. dev.'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/segantin/miniconda3/envs/baby_neutronics/lib/python3.9/site-packages/tables/attributeset.py:459: NaturalNameWarning: object name is not a valid Python identifier: 'Shield depth (cm)_kind'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-

Let's verify that we stored something in our file:

In [6]:
openmc_file = osb.ResultsFromDatabase(filename='openmc_fendl32b.h5', path='example_database')
openmc_file.get_tally_dataframe(tally_name='rr_onaxis1_nb93')

array([( 0, b'0.25', b'c0eb4a8954a405', 1.22189201e-03, 1.63312534e-07),
       ( 1, b'12.95', b'c0eb4a8954a405', 1.40189803e-04, 5.46831183e-08),
       ( 2, b'25.95', b'c0eb4a8954a405', 5.00503709e-05, 3.21662356e-08),
       ( 3, b'38.65', b'c0eb4a8954a405', 2.39532182e-05, 1.98394803e-08),
       ( 4, b'43.82', b'c0eb4a8954a405', 1.85102692e-05, 1.65171604e-08),
       ( 5, b'46.35', b'c0eb4a8954a405', 1.23130039e-05, 1.35659245e-08),
       ( 6, b'53.3', b'c0eb4a8954a405', 3.62911963e-06, 9.24821360e-09),
       ( 7, b'60.05', b'c0eb4a8954a405', 1.02807896e-06, 4.51423650e-09),
       ( 8, b'66.9', b'c0eb4a8954a405', 2.85057194e-07, 2.16584913e-09),
       ( 9, b'73.9', b'c0eb4a8954a405', 8.09682264e-08, 1.23753654e-09),
       (10, b'80.6', b'c0eb4a8954a405', 2.26181498e-08, 5.67881295e-10),
       (11, b'87.25', b'c0eb4a8954a405', 7.79119101e-09, 3.87065143e-10),
       (12, b'91.65', b'c0eb4a8954a405', 3.30897239e-09, 2.54791800e-10)],
      dtype=[('index', '<i8'), ('Shield de

In [7]:
openmc_file.print_all_info()

When: November 2023
 Where: MIT-PSFC
 Code version:openmc-0.13.3
 XS library: FENDL-3.2b
Literature: n/a

